<a href="https://colab.research.google.com/github/Rochitha-prabhuram/Digital-EHR/blob/main/Finalcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -qq
!apt-get install -y poppler-utils tesseract-ocr

!pip install -q pdf2image pytesseract opencv-python-headless
!pip install -q spacy medspacy
!pip install -q pandas numpy regex


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 33 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (328 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.0

In [2]:
import cv2
import pytesseract
import numpy as np
import pandas as pd
import regex as re
import json

from pdf2image import convert_from_path
import spacy
import medspacy


In [24]:
# Base English NLP - Load without the default parser and NER to avoid conflicts with medspacy's components
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Medical extensions (rule-based, stable)
nlp = medspacy.load(nlp)

In [7]:
PDF_PATH = /content/Input doc.pdf

SyntaxError: invalid syntax (ipython-input-1774332139.py, line 1)

Please upload your `Input doc.pdf` file using the following code cell. Once uploaded, its path in Colab will be `/content/Input doc.pdf`.

In [8]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving Input doc.pdf to Input doc (1).pdf
User uploaded file "Input doc (1).pdf" with length 2064696 bytes


In [18]:
PDF_PATH = "/content/Input doc (1).pdf"

In [19]:
pages = convert_from_path(PDF_PATH, dpi=300)
print(f"Total pages detected: {len(pages)}")

Total pages detected: 20


In [11]:
def preprocess_image(pil_img):
    img = np.array(pil_img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.adaptiveThreshold(
        blur, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 31, 2
    )
    return thresh


In [12]:
def extract_text(img):
    data = pytesseract.image_to_data(
        img,
        output_type=pytesseract.Output.DICT,
        config="--psm 6"
    )

    words = []
    for i in range(len(data["text"])):
        if int(data["conf"][i]) > 50:
            words.append(data["text"][i])

    return " ".join(words)


In [13]:
page_texts = {}

for i, page in enumerate(pages):
    processed = preprocess_image(page)
    text = extract_text(processed)
    page_texts[f"page_{i+1}"] = text


In [14]:
def classify_page(text):
    t = text.lower()
    if "in-patient record" in t or "age / sex" in t:
        return "demographics"
    if "t.p.r" in t or "pulse" in t:
        return "vitals"
    if "tab" in t or "inj" in t:
        return "medications"
    if "investigation" in t or "hb" in t:
        return "labs"
    if "discharge" in t or "summary" in t:
        return "discharge"
    return "other"


In [15]:
def extract_patient_info(text):
    data = {}

    name = re.search(r"Name\s*:\s*([A-Z ]+)", text)
    age = re.search(r"Age\s*/\s*Sex\s*:\s*(\d+)", text)
    doa = re.search(r"D\.O\.A\s*:\s*([\d/]+)", text)
    diagnosis = re.search(r"Diagnosis\s*:\s*(.+)", text)

    if name:
        data["name"] = name.group(1).title()
    if age:
        data["age"] = int(age.group(1))
    if doa:
        data["date_of_admission"] = doa.group(1)
    if diagnosis:
        data["diagnosis"] = diagnosis.group(1).strip()

    return data


In [16]:
def extract_medical_entities(text):
    doc = nlp(text)
    entities = set()

    for ent in doc.ents:
        entities.add(ent.text)

    return list(entities)


In [23]:
output = {
    "patient": {},
    "entities": [],
    "pages": []
}

for page, text in page_texts.items():
    category = classify_page(text)

    if category == "demographics":
        output["patient"].update(extract_patient_info(text))

    output["entities"].extend(extract_medical_entities(text))
    output["pages"].append({
        "page": page,
        "category": category
    })

2026-01-11 05:38:35.622 | DEBUG    | PyRuSH.PyRuSHSentencizer:predict:100 - [cpredict_split_gaps|call_id=21] [doc 0] Token 0 '&' marked as sentence start (span begin)
2026-01-11 05:38:35.623 | DEBUG    | PyRuSH.PyRuSHSentencizer:predict:100 - [cpredict_split_gaps|call_id=21] [doc 0] Token 47 '"' marked as sentence start (span end next token)
2026-01-11 05:38:35.624 | DEBUG    | PyRuSH.PyRuSHSentencizer:predict:100 - [cpredict_split_gaps|call_id=21] [doc 0] Token 47 '"' marked as sentence start (span begin)
2026-01-11 05:38:35.626 | DEBUG    | PyRuSH.PyRuSHSentencizer:predict:100 - [cpredict_split_gaps|call_id=21] [doc 0] Token 121 '2' marked as sentence start (span end next token)
2026-01-11 05:38:35.627 | DEBUG    | PyRuSH.PyRuSHSentencizer:predict:100 - [cpredict_split_gaps|call_id=21] [doc 0] Token 121 '2' marked as sentence start (span begin)
2026-01-11 05:38:35.628 | DEBUG    | PyRuSH.PyRuSHSentencizer:predict:100 - [cpredict_split_gaps|call_id=21] Token/tag mapping: [(&, True), (

In [26]:
output["entities"] = list(set(output["entities"]))

with open("ehr_extracted.json", "w") as f:
    json.dump(output, f, indent=4)

print("✅ Extraction complete. File saved as ehr_extracted.json")


✅ Extraction complete. File saved as ehr_extracted.json


In [40]:
import json

with open("ehr_extracted.json", "r") as f:
    extracted_data = json.load(f)

print(json.dumps(extracted_data, indent=4))

{
    "patient": {},
    "entities": [],
    "pages": [
        {
            "page": "page_1",
            "category": "other"
        },
        {
            "page": "page_2",
            "category": "labs"
        },
        {
            "page": "page_3",
            "category": "discharge"
        },
        {
            "page": "page_4",
            "category": "medications"
        },
        {
            "page": "page_5",
            "category": "other"
        },
        {
            "page": "page_6",
            "category": "other"
        },
        {
            "page": "page_7",
            "category": "other"
        },
        {
            "page": "page_8",
            "category": "other"
        },
        {
            "page": "page_9",
            "category": "other"
        },
        {
            "page": "page_10",
            "category": "other"
        },
        {
            "page": "page_11",
            "category": "other"
        },
        {
          

In [34]:
def normalize_text(text):
    text = text.replace("|", " ")
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"(hospital|medical college|road|phone|coimbatore)", "", text, flags=re.I)
    return text.strip()


In [35]:
text = normalize_text(text)


In [36]:
PROCEDURE_MAP = {
    "debridement": "Wound debridement",
    "incision": "Incision and drainage",
    "drainage": "Incision and drainage",
    "surgery": "Surgical procedure",
    "operated": "Surgical procedure",
    "suturing": "Wound suturing",
    "dressing": "Wound dressing",
}


In [28]:
# Common clinical procedure keywords
PROCEDURE_KEYWORDS = [
    "surgery", "surgical", "operation", "operated",
    "debridement", "incision", "drainage",
    "suturing", "dressing", "amputation",
    "catheterization", "biopsy"
]

# Investigation keywords
INVESTIGATION_KEYWORDS = [
    "hb", "wbc", "rbc", "platelet",
    "x-ray", "ct", "mri", "usg", "ultrasound",
    "ecg", "echo", "blood sugar", "creatinine"
]

# Medication indicators
MED_PREFIX = ["tab", "inj", "syp", "cap", "ointment", "cream"]


In [37]:
def extract_procedures(text):
    found = set()
    lower = text.lower()

    for key, label in PROCEDURE_MAP.items():
        if key in lower:
            found.add(label)

    return list(found)



In [30]:
def extract_investigations(text):
    investigations = set()
    lower_text = text.lower()

    for test in INVESTIGATION_KEYWORDS:
        if test in lower_text:
            investigations.add(test.upper())

    return list(investigations)


In [31]:
def extract_medications(text):
    medications = []

    lines = re.split(r"\n|,", text)

    for line in lines:
        line_lower = line.lower()
        for prefix in MED_PREFIX:
            if line_lower.strip().startswith(prefix):
                med = {}

                med["raw_text"] = line.strip()

                # Drug name
                name_match = re.search(r"(tab|inj|cap|syp)\s+([a-zA-Z]+)", line_lower)
                if name_match:
                    med["drug_name"] = name_match.group(2).title()

                # Dose
                dose_match = re.search(r"(\d+\s*(mg|ml))", line_lower)
                if dose_match:
                    med["dose"] = dose_match.group(1)

                # Frequency
                freq_match = re.search(r"(bd|tid|od|hs)", line_lower)
                if freq_match:
                    med["frequency"] = freq_match.group(1).upper()

                medications.append(med)

    return medications


In [41]:
MED_REGEX = re.compile(r"(tab|inj|cap|syp|ointment|cream)\s+([a-zA-Z]+(?:\s+[a-zA-Z]+)*)(?:\s+(\d+(?:\.\d+)?\s*(?:mg|ml|units)))?(?:.*?(bd|tid|od|hs|q\d+h))?")

def extract_medications(text):
    meds = []

    for match in MED_REGEX.finditer(text):
        med = {
            "form": match.group(1).upper(),
            "drug_name": match.group(2).title(),
            "dose": match.group(3) if match.group(3) else None,
            "frequency": match.group(5).upper() if match.group(5) else None
        }
        meds.append(med)

    return meds

In [32]:
final_output = {
    "patient": {},
    "procedures": [],
    "medications": [],
    "investigations": [],
    "page_summary": []
}

for page, text in page_texts.items():
    category = classify_page(text)

    if category == "demographics":
        final_output["patient"].update(extract_patient_info(text))

    if category in ["discharge", "other"]:
        final_output["procedures"].extend(extract_procedures(text))

    if category == "medications":
        final_output["medications"].extend(extract_medications(text))

    final_output["investigations"].extend(extract_investigations(text))

    final_output["page_summary"].append({
        "page": page,
        "category": category
    })


In [42]:
final_output = {
    "patient": {},
    "procedures": [],
    "medications": [],
    "investigations": [],
    "page_summary": []
}

for page, text in page_texts.items():
    text = normalize_text(text)
    category = classify_page(text)

    if category == "demographics":
        final_output["patient"].update(extract_patient_info(text))

    final_output["procedures"].extend(extract_procedures(text))
    final_output["medications"].extend(extract_medications(text))
    final_output["investigations"].extend(extract_investigations(text))

    final_output["page_summary"].append({
        "page": page,
        "category": category
    })

# Deduplicate
final_output["procedures"] = list(set(final_output["procedures"]))
final_output["investigations"] = list(set(final_output["investigations"]))

In [44]:
final_output["procedures"] = list(set(final_output["procedures"]))
final_output["investigations"] = list(set(final_output["investigations"]))

with open("ehr_clinical_insights.json", "w") as f:
    json.dump(final_output, f, indent=4)

print("✅ Clinical insights extracted successfully")

✅ Clinical insights extracted successfully
